In [ ]:
import os

if "notebooks" in os.getcwd():
    # Change the current working directory
    os.chdir("..")

### Test Function Calling

In [ ]:
from model.conversation import Conversation
from modules.chatbot import ChatBot

chatbot = ChatBot()
conversation = Conversation(chatbot.system_message)

conversation.add_turn("user", "Can you add the following questions as my daily questions: 1. What made me happy, 2. What made me sad")

